## DICOM to Video

This notebook demonstrates how to convert a Digital Imaging and Communications in Medicine (DICOM) file to one or more videos.

This example uses:
- `pydicom` library for reading and parsing DICOM files.
- `cv2` (installed as `opencv-python`) for image processing, color space conversions.
- `numpy` for numerical operations and handling arrays, useful to manipulate image pixel data.

In [1]:
# Imports
import cv2
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut, convert_color_space, apply_modality_lut
from pathlib import Path

In [2]:
# DICOM source file (`.dcm` extension).
SRC_PATH = "./example/input/sample_monochrome2_multiframe.dcm"

# Output path to store each frame.
VIDEO_PATH = Path("./example/output/dicom_video.mp4")
FPS = 10

### Load DICOM file and its image data

In [3]:
ds = pydicom.dcmread(SRC_PATH)
pixel_array = ds.pixel_array

### Apply Modality LUT

DICOMs may contain a Modality Look-Up Table (LUT) to transform **raw data** from imaging devices into standardized units (i.e. Hounsfield units which quantify tissue density) also known as "modality-specific values" to ensure consistency regardless of the device used to acquire the image.

In [4]:
if "ModalityLUTSequence" in ds or ("RescaleSlope" in ds and "RescaleIntercept" in ds):
    pixel_array = apply_modality_lut(pixel_array, ds)
else:
    slope = ds.get("RescaleSlope", 1.0)
    intercept = ds.get("RescaleIntercept", 0.0)
    pixel_array = pixel_array * slope + intercept

### Apply VOI LUT or Window Level

DICOMs may contain a VOI LUT (Value of Interest Look-Up Table) to transform modality-specific values into pixel values that best reflect the desired content or area of interest. For example, to enhance specific features in medical images to highlighting tumors or blood vessels.

In [5]:
pixel_array = pixel_array.astype(np.int32)
pixel_array = apply_voi_lut(pixel_array, ds)

### Convert source image type to RGB color space

The DICOM file may contain an image that is grayscale (MONOCHROME1 or MONOCHROME2), RGB, YBR, or palette color. The code below converts the source image from its respective color space to RGB color space.

- The "PALETTE COLOR" type is not handled, as it is more complicated and out of the scope of this example notebook.
- The "MONOCHROME2" type does not need conversion because pixel values are already dark-to-bright in ascending order.
- And the "RGB" type is already in the target color space.

In [6]:
image_type = ds.get("PhotometricInterpretation", "MONOCHROME2")
if image_type == "MONOCHROME1":
    pixel_array = np.amax(pixel_array) - pixel_array
elif image_type in ("RGB", "YBR_FULL", "YBR_FULL_422"):
    pixel_array = convert_color_space(pixel_array, image_type, "RGB")
    pixel_array = cv2.cvtColor(pixel_array, cv2.COLOR_RGB2BGR)
    
pixel_array = cv2.normalize(pixel_array, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

### Save DICOM frames to the output folder

In [7]:
frames = [frame for frame in pixel_array]

# Determine frame size
height, width = frames[0].shape[:2]
print("Number of frames:", len(frames))

Number of frames: 96


In [8]:
# Setup Video Writer
VIDEO_PATH.parent.mkdir(parents=True, exist_ok=True)
is_color = True if len(frames[0].shape) == 3 else False
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

is_color = False if len(frames[0].shape) == 2 else True
video_writer = cv2.VideoWriter(str(VIDEO_PATH), fourcc, FPS, (width, height), isColor=is_color)

In [9]:
# Write Frames to Video
for i, frame in enumerate(frames):
    if is_color and len(frame.shape) == 2:
        frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)
    elif not is_color and len(frame.shape) == 3:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    video_writer.write(frame)

video_writer.release()
print(f"File size (bytes):{VIDEO_PATH.stat().st_size} saved to: {VIDEO_PATH}")

File size (bytes):359509 saved to: example/output/dicom_video.mp4
